## Лабораторная работа №2. Сортировка слиянием. Метод декомпозиции

In [2]:
# Хочу строгую типизацию
import typing as tp

### Задание 1

Реализуйте сортировку слиянием

In [3]:
def merge(data: tp.List[int], l: int, p: int, r: int) -> None:
    l_part, r_part = data[l:p], data[p:r + 1]
    l_i, r_i, i = 0, 0, l
    while l_i < len(l_part) or r_i < len(r_part):
        if l_i < len(l_part) and (r_i >= len(r_part) or l_part[l_i] < r_part[r_i]):
            data[i] = l_part[l_i]
            l_i += 1
        else:
            data[i] = r_part[r_i]
            r_i += 1
        i += 1

In [4]:
def merge_sort(data: tp.List[int]) -> tp.List[int]:
    def inner_merge_sort(data: tp.List[int], l: int, r: int) -> None:
        size = r - l + 1

        if size <= 1:
            return
        elif size == 2:
            data[l], data[r] = min(data[l], data[r]), max(data[l], data[r])
            return
        
        p = (r + l) // 2
        inner_merge_sort(data, l, p)
        inner_merge_sort(data, p + 1, r)
        merge(data, l, p + 1, r)

    inner_merge_sort(data, 0, max(len(data) - 1, 0))
    return data

In [5]:
# Проверка
print(merge_sort([5, 4, 3, 2, 1, 0]))
print(merge_sort([5, 4, 3, 2, 1]))
print(merge_sort([3, 2, 3]))
print(merge_sort([3, 2]))
print(merge_sort([]))

[0, 1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]
[2, 3, 3]
[2, 3]
[]


### Задание 2

Реализуйте сортировку слиянием с сохранением истории слияний
(Индекс начала области слияния, индекс конца области слияния, первый элемент области слияния, последний элемент области слияния)

In [6]:
def merge_sort_with_history(data: tp.List[int]) -> tp.Tuple[tp.List[int], tp.List[tp.Tuple[int, int, int, int]]]:
    history = []
    
    def inner_merge_sort_with_history(data: tp.List[int], l: int, r: int) -> None:
        size = r - l + 1

        if size <= 1:
            return
        elif size == 2:
            data[l], data[r] = min(data[l], data[r]), max(data[l], data[r])
            history.append((l + 1, r + 1, data[l], data[r]))
            return
        
        p = (r + l) // 2
        inner_merge_sort_with_history(data, l, p)
        inner_merge_sort_with_history(data, p + 1, r)
        merge(data, l, p + 1, r)
        history.append((l + 1, r + 1, data[l], data[r]))

    inner_merge_sort_with_history(data, 0, max(len(data) - 1, 0))
    return (data, history)

In [7]:
# Проверка
print(merge_sort_with_history([9, 7, 5, 8]))

([5, 7, 8, 9], [(1, 2, 7, 9), (3, 4, 5, 8), (1, 4, 5, 9)])


### Задание 3

Найдите количество инверсий в массиве

In [8]:
def count_inversions(data: tp.List[int]) -> int:
    def merge_with_inversions_count(a: tp.List[int], b: tp.List[int]) -> tp.Tuple[tp.List[int], int]:
        merged, counter = [], 0
        while len(a) > 0 and len(b) > 0:
            if a[0] > b[0]:
                merged.append(a.pop(0))
                counter += len(b)
            else:
                merged.append(b.pop(0))
        return (merged + a + b, counter)
    
    def merge_sort_with_inversions_count(data: tp.List[int]) -> tp.Tuple[tp.List[int], int]:
        if len(data) <= 1:
            return (data, 0)
        
        l = data[:len(data) // 2]
        r = data[len(data) // 2:]

        ls, lc = merge_sort_with_inversions_count(l)
        rs, rc = merge_sort_with_inversions_count(r)

        res, res_c = merge_with_inversions_count(ls, rs)
        return (res, lc + rc + res_c)
    
    return merge_sort_with_inversions_count(data)[1]

In [9]:
# Проверка
print(count_inversions([1, 2, 3, 4]))
print(count_inversions([4, 3, 2, 1]))
print(count_inversions([3, 2, 1]))
print(count_inversions([2]))
print(count_inversions([]))

0
6
3
0
0


### Задание 4

Реализуйте бинарный поиск
(Здесь я ищу наименьший индекс, куда нужно поставить элемент, чтобы массив остался отсортированным)

In [10]:
def lbin_search(data: tp.List[int], value: int) -> int:
    l, r = 0, len(data)

    while l < r:
        m = (r + l) // 2

        if data[m] > value:
            r = m
        elif data[m] < value:
            l = m + 1
        else:
            r = m
    
    return max(l, r)

In [11]:
# Проверка
print(lbin_search([0, 0, 0, 0, 5], 4))
print(lbin_search([4], 4))
print(lbin_search([], 4))
print(lbin_search([1, 2, 3, 4, 5, 6, 7], 7))
print(lbin_search([1, 2, 2, 2, 3, 3, 3, 4, 5], 4))
print(lbin_search([1, 2, 3, 4], 5))

4
0
0
6
7
4


### Задание 5

Понять, есть ли во входной последовательности элемент, который встречается более половины раз

In [12]:
def has_more_than_half_element(data: tp.List[int]) -> bool:
    if len(data) == 0:
        return False
    
    merge_sort(data)
    last, r = data[0], 0

    while True:
        l = r
        r = lbin_search(data, last + 1)
        last = data[l]
        size = r - l

        if size > (len(data) // 2):
            return True

        if r >= len(data):
            return False
        

In [13]:
# Проверка
print(has_more_than_half_element([2, 3, 9, 2, 2]))
print(has_more_than_half_element([1, 2, 3, 4]))


True
False


In [14]:
def has_more_than_half_element_on(data: tp.List[int]) -> bool:
    counter, element = 0, None

    for i in range(len(data)):
        if element is None:
            element = data[i]
            counter = 1
        elif element == data[i]:
            counter += 1
        else:
            counter -= 1
            if counter == 0:
                element = None

    num = 0
    for i in data:
        if i == element:
            num += 1

    return (num * 2) > len(data)

In [15]:
# Проверка
print(has_more_than_half_element_on([2, 3, 9, 2, 2]))
print(has_more_than_half_element_on([1, 2, 3, 4]))

True
False


### Задание 6

Реализуйте поиск подмассива (непрерывного) с максимальной суммой элементов

In [16]:
def find_max_subarray(data: tp.List[int]) -> tp.Tuple[int, int, int]:
    if len(data) == 0:
        return (0, 0, 0)

    def inner_find_max_subarray_middle(data: tp.List[int], l: int, r: int, p: int) -> tp.Tuple[int, int, int]:
        cl, ml, mli = data[p], data[p], p
        cr, mr, mri = data[p + 1], data[p + 1], p + 1

        for i in range(p - 1, l - 1, -1):
            cl += data[i]
            if cl > ml:
                ml, mli = cl, i

        for i in range(p + 2, r + 1):
            cr += data[i]
            if cr > mr:
                mr, mri = cr, i

        return (mli, mri, ml + mr)

    def inner_find_max_subarray(data: tp.List[int], l: int, r: int) -> tp.Tuple[int, int, int]:
        if l == r:
            return (l, r, data[l])
        m = (l + r) // 2

        ll, lr, ls = inner_find_max_subarray(data, l, m)
        rl, rr, rs = inner_find_max_subarray(data, m + 1, r)
        ml, mr, ms = inner_find_max_subarray_middle(data, l=l, r=r, p=m)

        if ls > rs and ls > ms:
            return (ll, lr, ls)
        elif rs > ls and rs > ms:
            return (rl, rr, rs)
        else:
            return (ml, mr, ms)

    return inner_find_max_subarray(data, 0, len(data) - 1)

In [17]:
# Проверка
print(find_max_subarray([13, -3, -25, 20, -3, -16, -23, 18, 20, -7, 12, -5, -22, 15, -4, 7]))

(7, 10, 43)


In [18]:
def find_max_subarray_on(data: tp.List[int]) -> tp.Tuple[int, int, int]:
    if len(data) == 0:
        return (0, 0, 0)
    
    res = data[0]
    res_l, res_r = 0, 0

    current_sum = 0

    min_sum = 0
    min_pos = -1

    for i in range(len(data)):
        current_sum += data[i]

        if current_sum - min_sum > res:
            res = current_sum - min_sum
            res_l = min_pos + 1
            res_r = i

        if current_sum < min_sum:
            min_sum = current_sum
            min_pos = i

    return (res_l, res_r, res)

In [19]:
# Проверка
print(find_max_subarray_on([13, -3, -25, 20, -3, -16, -23, 18, 20, -7, 12, -5, -22, 15, -4, 7]))

(7, 10, 43)


In [20]:
def find_max_subarray_kadan(data: tp.List[int]) -> tp.Tuple[int, int, int]:
    if len(data) == 0:
        return (0, 0, 0)
    
    res = data[0]
    res_l, res_r = 0, 0

    current_sum = 0
    min_sum_pos = -1

    for i in range(len(data)):
        current_sum += data[i]
        
        if current_sum > res:
            res = current_sum
            res_l = min_sum_pos + 1
            res_r = i

        if current_sum < 0:
            current_sum = 0
            min_sum_pos = i

    return (res_l, res_r, res)

In [21]:
# Проверка
print(find_max_subarray_kadan([13, -3, -25, 20, -3, -16, -23, 18, 20, -7, 12, -5, -22, 15, -4, 7]))

(7, 10, 43)


### Задание 8

Реализуйте умножение многочленов

In [103]:
from math import ceil

def sum_polinoms(a: tp.List[int], b: tp.List[int]) -> tp.List[int]:
    return [sum(i) for i in zip([0] * max(0, len(b) - len(a)) + a, [0] * max(0, len(a) - len(b)) + b)]

def sub_polinoms(a: tp.List[int], b: tp.List[int]) -> tp.List[int]:
    return [i[0] - i[1] for i in zip([0] * max(0, len(b) - len(a)) + a, [0] * max(0, len(a) - len(b)) + b)]

def multiply_polinoms(a: tp.List[int], b: tp.List[int]) -> tp.List[int]:
    if len(a) != len(b):
        prefix = [0] * abs(len(a) - len(b))
        return multiply_polinoms(
            (a) if len(a) > len(b) else (prefix + a),
            (b) if len(b) > len(a) else (prefix + b)
        )
    
    n = len(a)
    if n == 0:
        return []
    elif n == 1:
        return [a[0] * b[0]]
    
    a1, a2 = a[:n // 2], a[n // 2:]
    b1, b2 = b[:n // 2], b[n // 2:]

    m1, m2 = multiply_polinoms(a1, b1), multiply_polinoms(a2, b2)
    
    a_sum = sum_polinoms(a1, a2)
    b_sum = sum_polinoms(b1, b2)
    
    m3 = multiply_polinoms(a_sum, b_sum)
    m4 = sub_polinoms(sub_polinoms(m3, m1), m2)

    return sum_polinoms(sum_polinoms(m1 + [0] * (2 * n - 2), m4 + [0] * ceil(n / 2.0)), m2)

In [104]:
# Проверка
print(multiply_polinoms([3], [5]))
print(multiply_polinoms([2, 5], [1, 2]))
print(multiply_polinoms([3, 2, 5], [5, 1, 2]))

[15]
[2, 9, 10]
[15, 13, 33, 9, 10]


### Задание 9

Реализуйте умножение матриц методом Штрассена

In [105]:
Matrix = tp.List[tp.List[int]]

In [126]:
def stupid_squares_multiply(a: Matrix, b: Matrix) -> Matrix:
    n = len(a)
    res = [[0] * n for _ in range(n)]

    for i in range(n):
        for j in range(n):
            for k in range(n):
                res[i][j] += (a[i][k] * b[k][j])

    return res

In [116]:
# Проверка
print(stupid_squares_multiply([[1]], [[1]]))
print(stupid_squares_multiply([[1, 2], [3, 4]], [[5, 6], [7, 8]]))

[[1]]
[[19, 22], [43, 50]]


In [117]:
def split_square(a: Matrix) -> tp.Tuple[Matrix, Matrix, Matrix, Matrix]:
    n = len(a)

    return (
        [i[:n // 2] for i in a[:n // 2]],
        [i[n // 2:] for i in a[:n // 2]],
        [i[:n // 2] for i in a[n // 2:]],
        [i[n // 2:] for i in a[n // 2:]],
    )

In [122]:
def sum_squares(a: Matrix, b: Matrix) -> Matrix:
    return [[sum(j) for j in zip(i[0], i[1])] for i in zip(a, b)]

def sub_squares(a: Matrix, b: Matrix) -> Matrix:
    func = lambda x: x[0] - x[1]
    return [[func(j) for j in zip(i[0], i[1])] for i in zip(a, b)]

In [124]:
def shtrassen_squares_multiply(m1: Matrix, m2: Matrix) -> Matrix:
    n = len(m1)

    if n <= 2 or n % 2 == 1:
        return stupid_squares_multiply(m1, m2)
    
    a, b, c, d = split_square(m1)
    e, f, g, h = split_square(m2)

    p1 = shtrassen_squares_multiply(a, sub_squares(f, h))
    p2 = shtrassen_squares_multiply(sum_squares(a, b), h)
    p3 = shtrassen_squares_multiply(sum_squares(c, d), e)
    p4 = shtrassen_squares_multiply(d, sub_squares(g, e))
    p5 = shtrassen_squares_multiply(sum_squares(a, d), sum_squares(e, h))
    p6 = shtrassen_squares_multiply(sub_squares(b, d), sum_squares(g, h))
    p7 = shtrassen_squares_multiply(sub_squares(a, c), sum_squares(e, f))

    tl = sum_squares(sub_squares(sub_squares(p5, p4), p2), p6)
    tr = sum_squares(p1, p2)
    bl = sum_squares(p3, p4)
    br = sub_squares(sub_squares(sum_squares(p1, p5), p3), p7)

    l, r = tl + bl, tr + br

    func = lambda x: x[0] + x[1]
    return [func(i) for i in zip(l, r)]

In [127]:
m = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 1, 2, 3], [4, 5, 6 ,7]]
print(shtrassen_squares_multiply(m, m))

[[28, 47, 47, 57], [48, 119, 119, 145], [44, 41, 56, 71], [111, 79, 101, 123]]
